In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True, sheet_name="SP100 Original")

In [3]:
df.head()

,AMT,COP,CVX,AXP,BAC,C,CB,CME,GS,JPM,...,TMO,UNH,AMZN,HD,LOW,MCD,NKE,SBUX,TGT,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2007-06-27,0.014124,0.018373,0.015768,0.012207,0.010291,0.015468,0.009848,0.014583,0.017107,0.013571,...,0.015377,0.013590,0.014877,0.011604,0.014249,0.008126,0.020990,0.012522,0.011658,0.013830
2007-06-28,0.015762,0.011954,0.011587,0.010778,0.011098,0.013562,0.009857,0.010666,0.014501,0.012888,...,0.011744,0.010659,0.017926,0.009785,0.010404,0.010648,0.015595,0.014401,0.009406,0.012848
2007-06-29,0.011944,0.014456,0.012986,0.013216,0.008582,0.012972,0.010098,0.012094,0.014643,0.013615,...,0.011072,0.015759,0.011421,0.014842,0.012574,0.011361,0.017018,0.014760,0.008964,0.013771
2007-07-02,0.009732,0.009799,0.008922,0.009994,0.006183,0.010277,0.006879,0.009909,0.010316,0.008367,...,0.009402,0.008731,0.012612,0.008851,0.007254,0.007875,0.012682,0.011086,0.009220,0.010368
2007-07-06,0.009956,0.007930,0.007381,0.009133,0.005680,0.007148,0.009542,0.018029,0.009381,0.007802,...,0.009832,0.008183,0.010760,0.009840,0.010354,0.007864,0.010880,0.008781,0.020747,0.011182


In [4]:
train=df.iloc[:int(len(df)*0.7)]

In [5]:
test=df.iloc[int(len(df)*0.7)-22:]

In [6]:
df.iloc[int(len(df)*0.7)]

AMT     0.008684
COP     0.011713
CVX     0.008553
AXP     0.015038
BAC     0.009973
          ...   
MCD     0.004191
NKE     0.011698
SBUX    0.007688
TGT     0.010126
TJX     0.007017
Name: 2017-04-20 00:00:00, Length: 80, dtype: float64

In [7]:
train=np.array(train)

In [8]:
train.shape

(2386, 80)

In [9]:
RV_d = []
for i in range(len(train)-1):
  RV_d.append(train[i].astype(float))

RV_w = []
for i in range(len(train)-5):
  RV_w.append(np.mean(train[i:i+5]).astype(float))

RV_m = []
for i in range(len(train)-22):
  RV_m.append(np.mean(train[i:i+22]).astype(float))

RV_d=RV_d[len(RV_d)-len(RV_m):]
RV_w=RV_w[len(RV_w)-len(RV_m):]

In [10]:
RV_D = np.zeros([len(RV_d),train.shape[1]])
for i in range(len(RV_d)):
  RV_D[i,:]=RV_d[i]

RV_W = np.zeros([len(RV_w),train.shape[1]])
for i in range(len(RV_w)):
  RV_W[i,:]=RV_w[i]

RV_M = np.zeros([len(RV_m),train.shape[1]])
for i in range(len(RV_m)):
  RV_M[i,:]=RV_m[i]

In [11]:
RV_actuals=train[22:,:]

In [12]:
RV_actuals.shape

(2364, 80)

In [13]:
test=np.array(test)

In [14]:
test.shape

(1045, 80)

In [15]:
  for i in range(len(test)-1):
    RV_d.append(test[i].astype(float))

  RV_w = []
  for i in range(len(test)-5):
    RV_w.append(np.mean(test[i:i+5]).astype(float))

  RV_m = []
  for i in range(len(test)-22):
    RV_m.append(np.mean(test[i:i+22]).astype(float))

  RV_d=RV_d[len(RV_d)-len(RV_m):]
  RV_w=RV_w[len(RV_w)-len(RV_m):]
  RV_D_test = np.zeros([len(RV_d),test.shape[1]])
  for i in range(len(RV_d)):
    RV_D_test[i,:]=RV_d[i]

  RV_W_test = np.zeros([len(RV_w),test.shape[1]])
  for i in range(len(RV_w)):
    RV_W_test[i,:]=RV_w[i]

  RV_M_test = np.zeros([len(RV_m),test.shape[1]])
  for i in range(len(RV_m)):
    RV_M_test[i,:]=RV_m[i]

In [16]:
forecasts=np.zeros([test.shape[0]-22,test.shape[1]])
for j in range(test.shape[1]):
  Data ={'Actual': RV_actuals[:,j],
  'D': RV_D[:,j],
  'W': RV_W[:,j],
  'M': RV_M[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  rv_scaled = sm.add_constant(rv_scaled)
  rv_scaled['Actual']=rv_scaled['Actual'].astype(float)
  rv_scaled['D']=rv_scaled['D'].astype(float)
  rv_scaled['W']=rv_scaled['W'].astype(float)
  rv_scaled['M']=rv_scaled['M'].astype(float)
  X = rv_scaled.drop("Actual", axis = 1)
  y = rv_scaled["Actual"]
  results = sm.OLS(y, X).fit()
  Data ={'D': RV_D_test[:,j],
  'W': RV_W_test[:,j],
  'M': RV_M_test[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  X_test = sm.add_constant(rv_scaled)
  forecasts[:,j]= results.predict(X_test)

In [17]:
forecasts.shape

(1023, 80)

In [18]:
forecasts[0,:]

array([0.00692693, 0.01223739, 0.00768858, 0.00648956, 0.01029492,
       0.00916709, 0.00490023, 0.00756315, 0.0099795 , 0.00747045,
       0.01226608, 0.00923341, 0.00916405, 0.00816076, 0.00722254,
       0.00974343, 0.00594828, 0.00561192, 0.00572033, 0.00500484,
       0.00912295, 0.00571901, 0.0064204 , 0.00717041, 0.00729849,
       0.00594057, 0.00744224, 0.00887952, 0.00691947, 0.00614391,
       0.00608169, 0.00601917, 0.00624495, 0.01100931, 0.00759979,
       0.00726811, 0.00656553, 0.00666573, 0.01549955, 0.01362448,
       0.0066418 , 0.00924821, 0.00558265, 0.00826561, 0.00681422,
       0.00574941, 0.00545403, 0.00537893, 0.00502824, 0.00472005,
       0.00567321, 0.00657959, 0.00657594, 0.00604628, 0.01003024,
       0.00774495, 0.00647609, 0.00788104, 0.00946653, 0.00830017,
       0.00620713, 0.00628969, 0.0091043 , 0.01261463, 0.00556955,
       0.01080233, 0.00602223, 0.00675814, 0.00676868, 0.0065365 ,
       0.0065876 , 0.00881293, 0.00833579, 0.00624989, 0.00748

In [19]:
df1=df.iloc[int(len(df)*0.7):]

In [20]:
df1.head()

,AMT,COP,CVX,AXP,BAC,C,CB,CME,GS,JPM,...,TMO,UNH,AMZN,HD,LOW,MCD,NKE,SBUX,TGT,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2017-04-20,0.008684,0.011713,0.008553,0.015038,0.009973,0.010190,0.006542,0.007843,0.009500,0.007578,...,0.006517,0.008160,0.007141,0.005866,0.006494,0.004191,0.011698,0.007688,0.010126,0.007017
2017-04-21,0.005333,0.013754,0.009606,0.009791,0.011717,0.009124,0.005080,0.008150,0.009581,0.008267,...,0.006502,0.006875,0.004083,0.006627,0.007835,0.004260,0.007366,0.006447,0.008618,0.006344
2017-04-24,0.007305,0.009744,0.007956,0.006219,0.014207,0.009040,0.004838,0.009033,0.008960,0.008200,...,0.006311,0.006555,0.004789,0.005144,0.007185,0.007506,0.007695,0.006543,0.008466,0.006292
2017-04-25,0.005686,0.009516,0.006491,0.005460,0.011123,0.009362,0.006507,0.008783,0.008505,0.008336,...,0.006162,0.007088,0.005912,0.009816,0.009604,0.013330,0.006887,0.011564,0.005370,0.004877
2017-04-26,0.006191,0.016867,0.009998,0.007953,0.014055,0.011115,0.011037,0.008396,0.010142,0.008328,...,0.016428,0.006435,0.007461,0.007735,0.007850,0.007920,0.007706,0.007522,0.008771,0.006573


In [21]:
for i in range(len(df1.columns)):
  df1[df1.columns[i]]=forecasts[:,i]

<ipython-input-21-8a232fbfd1e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[df1.columns[i]]=forecasts[:,i]


In [22]:
df1.to_excel("HAR Forecasts.xlsx")

In [23]:
test=df.iloc[int(len(df)*0.7):]
test=np.array(test)
RMSE=np.sqrt(np.sum((test-forecasts)**2)/(test.shape[0]*test.shape[1]))
MAE=(np.sum(abs(test-forecasts))/(test.shape[0]*test.shape[1]))
QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
MAPE=np.sum(abs(test-forecasts)/(test))/(test.shape[0]*test.shape[1])
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

RMSE: 0.0037569319213711706 | QLIKE: 0.03242962182000285 | MAE: 0.002471743470290965 | MAPE: 0.1985684662663057
